In [ ]:
#r "D:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
// #r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Solution.NSECommon;
using BoSSS.Application.XNSEC;
Init();

Error: System.ApplicationException: Already called.
   at BoSSS.Application.BoSSSpad.BoSSSshell.InitTraceFile() in D:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 170
   at BoSSS.Application.BoSSSpad.BoSSSshell.Init() in D:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 97
   at Submission#29.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
using BoSSS.Foundation.Quadrature;
using BoSSS.Foundation.XDG;
using MathNet.Numerics;
using MathNet.Numerics.Interpolation;
using MathNet.Numerics.RootFinding;

In [ ]:
BoSSSshell.WorkflowMgm.Init("HBFS_ManualHomotopy_ERSweep");
// BoSSSshell.WorkflowMgm.Init("HBFS_ManualHomotopy_VariableDensity");

In [ ]:
var myDb = BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.DefaultDatabase;


Creation of a spline along desired edge. Used later for interpolation of variables.

In [ ]:
// Creates a Spline on direction "d" at a given edge "em" of the field "field"
static public CubicSpline SplineOnEdge(EdgeMask em, DGField field, out double lower_Bound, out double upper_Bound, double offset = 0.0, int d = 0){

    var grd = field.GridDat;

    List<double> nodes = new List<double>();
    List<double> values = new List<double>();

    EdgeQuadrature.GetQuadrature(new int[] { 1 }, grd,
        // (new EdgeQuadratureScheme(true, em)).Compile(grd, 0),
        (new EdgeQuadratureScheme(true, em)).Compile(grd, field.Basis.Degree * 2),
        delegate (int i0, int Length, QuadRule QR, MultidimensionalArray EvalResult) {        

            MultidimensionalArray DummyIN = MultidimensionalArray.Create(Length, QR.NoOfNodes);
            MultidimensionalArray DummyOT = MultidimensionalArray.Create(Length, QR.NoOfNodes);        

            MultidimensionalArray GlobalNodes = MultidimensionalArray.Create(Length, QR.NoOfNodes, 2);        
            
            if(field is XDGField xField){
                xField.GetSpeciesShadowField("A").EvaluateEdge(i0, Length, QR.Nodes, DummyIN, DummyOT, null, null, null, null, 0, 0.0); 
            } else{
                field.EvaluateEdge(i0, Length, QR.Nodes, DummyIN, DummyOT, null, null, null, null, 0, 0.0); 
            }  

            for(int i = 0; i < Length; i++){  
                int iTrafo = ((GridData)grd).Edges.Edge2CellTrafoIndex[i0+i, 0];
                NodeSet volNodeSet = QR.Nodes.GetVolumeNodeSet(grd, iTrafo);
                int jCell = ((GridData)grd).Edges.CellIndices[i0+i, 0];
                grd.TransformLocal2Global(volNodeSet, jCell, 1, GlobalNodes, i);          
                int K = QR.NoOfNodes;
                for(int k = 0; k < K; k++){
                    nodes.Add(GlobalNodes[i, k, d]);
                    values.Add(DummyIN[i, k] - offset);
                }
            }

        },delegate (int i0, int Length, MultidimensionalArray ResultsOfIntegration) {
        }).Execute(); 

    lower_Bound = nodes.Min();
    upper_Bound = nodes.Max();

    return CubicSpline.InterpolateAkima(nodes.ToArray(), values.ToArray());
}



In [ ]:
// Finds the position where the specified field assumes the given value
static public double[] PosOfValueOnEdge(double value, EdgeMask em, DGField field, int d = 0) {
    var Spline = SplineOnEdge(em, field, out double lB, out double uB, value, d);
    int numberofintervals = 30;
    double[] intervalnodes = GenericBlas.Linspace(lB, uB, numberofintervals);
    List<double> roots = new List<double>();
    for (int i = 0; i < numberofintervals - 1; i++) {
        double root = -1111;
        bool found = Bisection.TryFindRoot(t => Spline.Interpolate(t), intervalnodes[i], intervalnodes[i + 1], 1e-12, 1000, out root);
        if (found)
            roots.Add(root);
    }
    return roots.ToArray();
}

 Calculation of Local Nusselt Number $\text{Nu} := \frac{S}{T0-T1}\vec{n}\cdot \nabla T$


In [ ]:
static public double[] LocalNusseltNumber(ITimestepInfo ts, double[] coordinates, double deltaT, int dirGradient, int dirSpline) {
    DGField Temperature       = ts.Fields.Where(f => f.Identification == "Temperature").SingleOrDefault();
    var grd = (GridData)Temperature.GridDat;
    var GradT_d = (DGField)Temperature.Clone();
    GradT_d.Clear();
    GradT_d.Derivative(1.0, Temperature, dirGradient);

    if(dirGradient == dirSpline){
    throw new Exception();
}
    EdgeMask em = new EdgeMask(grd, X => X[1] + 0.0 < 1e-12); // edge hardcoded
    var spline = SplineOnEdge(em, GradT_d, out double _a, out double  _b, d:dirSpline);
    
    double[] Nusselt = new double[coordinates.Length];
    for(int i = 0; i < coordinates.Length; i++){
        double GradT = spline.Interpolate(coordinates[i]);    
        Nusselt[i] = -1.0*GradT/deltaT;
    }   
    return Nusselt;
}

Calculation of Local Darcy frictionfactor $\text{f}_D := -\frac{8\nu}{(\hat{u}/2)^2}\vec{n}\cdot \nabla u_1$


In [ ]:
static public double[] LocalDarcyFactor(ITimestepInfo ts, double[] coordinates, int dirGradient, int dirSpline, double Re, double factor2 ) {
    DGField VelocityX       = ts.Fields.Where(f => f.Identification == "VelocityX").SingleOrDefault();

        
    double nu = 15.52e-6;// Kinematic viscosity of air at 25°c, m2/s
    double S = 15.0/1000; // Reference length according to the paper of XIA, meters
    double vel = Re*nu/S; // mean velocity, m/s
    // Console.WriteLine("Re:" + Re);
    // Console.WriteLine("vel:" + vel);
    //  double ReDimFactor =vel/S*0+1;

    // Console.WriteLine("ReDimFactor:" + ReDimFactor);
    var grd = (GridData)VelocityX.GridDat;
    var GradVelX_d = (DGField)VelocityX.Clone();
    GradVelX_d.Clear();

 
    GradVelX_d.Derivative(1.0 , VelocityX, dirGradient);

    if(dirGradient == dirSpline){
    throw new Exception();
}
    EdgeMask em = new EdgeMask(grd, X => X[1] + 0.0 < 1e-12); // 
    var spline = SplineOnEdge(em, GradVelX_d, out double _a, out double  _b, d:dirSpline);

    double factor = 4*nu/(vel*vel/4.0);

    double[] fd = new double[coordinates.Length];
    for(int i = 0; i < coordinates.Length; i++){
        double GradVelX_dVal = spline.Interpolate(coordinates[i]);    
        fd[i] = factor*GradVelX_dVal*factor2 ;
    }   
    return fd;
}


The position(s) of deatachment occurs exactly at the point(s) where the friction factor is equal to zero $f_d(\vec{x})=0$

In [ ]:
static public double[] GetPositionOfReattachment(ITimestepInfo ts, int dirGradient, int dirSpline, double ER, bool top ) {
    DGField VelocityX       = ts.Fields.Where(f => f.Identification == "VelocityX").SingleOrDefault();
    var grd = (GridData)VelocityX.GridDat;
    var GradVelX_d = (DGField)VelocityX.Clone();
    GradVelX_d.Clear();
    GradVelX_d.Derivative(1.0, VelocityX, dirGradient);

    double S = 1.0;
    double H = S+S/(ER-1.0);
    double ywall = top == true? H : 0.0;
    EdgeMask em = new EdgeMask(grd, X => Math.Abs(X[1] - ywall) < 1e-12); 
    double[] x =   PosOfValueOnEdge(0.0, em, GradVelX_d, d:0); // 

    
    return x ;
}

Now we calculate for each Session where Re = {400,700,100} for each ER = {1.5,2.0,2.5} the friction coefficient and Nusselt number

In [ ]:
// double[] Reynolds = new double[]{400,700};
// double[] ER = new double[]{1.5,2.0};

// double[] Reynolds = new double[]{400,700,1000};
//  double[] Reynolds = new double[]{100,150,200,250,300,350,400};
double[] Reynolds = new double[]{200,250,300,350,400};

// var Reynolds = ((XNSEC_Control)myDb.Sessions[0].GetControl()).SelfDefinedHomotopyArray;


// double[] Reynolds = new double[]{400};
double[] ER = new double[]{2.0};

// double[] ER = new double[]{1.9423};
// double[] ER = new double[]{1.5,2.0,2.5};

In [ ]:
double[] xCoordinates  = GenericBlas.Linspace(0,20, 1000);

Dictionary<(double,double), double[]> cfs = new Dictionary<(double,double), double[]>();
Dictionary<(double,double), double[]> Nusselts = new Dictionary<(double,double), double[]>();
Dictionary<(double,double), double[]> ReattachmentPositions = new Dictionary<(double,double),  double[]>();
Dictionary<(double,double), double[]> ReattachmentPositionsTOP = new Dictionary<(double,double), double[]>();


foreach(double er in ER){
    // load timestep. The database contains three sessions, each one corresponds to a ER
    var sess = myDb.Sessions.Where(s => s.Tags.Pick(0) == er.ToString()).SingleOrDefault();

    var control =  (XNSEC_Control)sess.GetControl();
    var ReynoldsArray = control.SelfDefinedHomotopyArray;
    Dictionary<int, double> ReynoldsDictionary = new Dictionary<int, double>();
    for(int i = 0; i <ReynoldsArray.Length;i++ ){
        ReynoldsDictionary.Add(i, ReynoldsArray[i]);
    }


    foreach(double Re in Reynolds){        
        int indexOfReynolds = ReynoldsDictionary.FirstOrDefault(x => x.Value == Re).Key;
        var timestep = sess.Timesteps[indexOfReynolds+1];

        // Calculation of the local Nusselt number          
        double tin = 10 +273;
        double thot = 40+273;
        double tref = tin;
        double TIN = tin/tref;
        double THOT = thot/tref;
        var nuss = LocalNusseltNumber(timestep,xCoordinates, -(TIN-THOT), 1, 0 );
        Nusselts.Add((er,Re),nuss);

        //Calculation of the darcy friction factor
        double[] cf = LocalDarcyFactor(timestep, xCoordinates, 1,0,Re,1.0);
        cfs.Add((er,Re),cf);

        // // Reattachment positions 
        double[] X = GetPositionOfReattachment(timestep,1,0,er,top:false); 
        ReattachmentPositions.Add((er,Re),X);

        double[] XTop = GetPositionOfReattachment(timestep,1,0,er, top:true);
        ReattachmentPositionsTOP.Add((er,Re),XTop);

    }   
}


In [ ]:
ReattachmentPositionsTOP

key,value
"(2, 200)",[ ]
"(2, 250)","[ 8.51916343540012, 12.329609712614705 ]"
"(2, 300)","[ 8.938770309901487, 15.111056992206267 ]"
"(2, 350)","[ 9.395943066476896, 17.57079780908193 ]"
"(2, 400)","[ 9.854436725989947, 19.81351305071657 ]"


Plot for cf

In [ ]:
LineColors[] allColors = Enum.GetValues(typeof(LineColors)).Cast<LineColors>().ToArray();
PointTypes[] myPointTypes = new PointTypes[] { PointTypes.Diamond, PointTypes.Box, PointTypes.LowerTriangle, PointTypes.OpenLowerTriangle, };
LineColors[] myCollors = new LineColors[] { LineColors.Red, LineColors.Orange, LineColors.Blue };

Plot2Ddata[,] PlotTableCfs = new Plot2Ddata[1, 3];
Plot2Ddata[,] PlotTableNusselts = new Plot2Ddata[1, 3];
int counter = 0;
double[] ReynoldsFromPaper = new double[]{400,700,1000};
foreach (double Re in ReynoldsFromPaper) {
    var plotcf = new Plot2Ddata();
    var plotnuss = new Plot2Ddata();

    // BoSSS Data
    int cnt = 0;
    foreach (double er in ER) {
        var fmt = new PlotFormat();
        fmt.Style = Styles.Lines;

        fmt.LineColor = myCollors[cnt];
        plotcf.AddDataGroup("ER=" + er, xCoordinates, cfs[(er, Re)], fmt);
        plotnuss.AddDataGroup("ER=" + er, xCoordinates, Nusselts[(er, Re)], fmt);

        // Add reference data
        // Data avaliable for Re 400 700 1000 and ER 1.5 2.0 2.5
        if(er == 1.5 || er == 2.0 || er == 2.5){
        MultidimensionalArray[] ReferenceDatacf = new MultidimensionalArray[1];
        MultidimensionalArray[] ReferenceDataNu = new MultidimensionalArray[1];
        string CurrentDocDir = Directory.GetCurrentDirectory();
        string erstr = ((int)(er * 10)).ToString();

        string path1 = Path.Combine(CurrentDocDir, @"CF_RE" + Re + "_ER" + erstr + ".txt"); // CF_RE400_ER15 Data avaliable for Re 400 700 1000 and ER 1.5 2.0 2.5
        string path2 = Path.Combine(CurrentDocDir, @"NU_RE" + Re + "_ER" + erstr + ".txt"); // CF_RE400_ER15
        ReferenceDatacf[0] = IMatrixExtensions.LoadFromTextFile(path1);
        ReferenceDataNu[0] = IMatrixExtensions.LoadFromTextFile(path2);
        var fmt4 = new PlotFormat();
        fmt4.Style = Styles.Points;
        // fmt.Style = Styles.Lines;

        fmt4.PointType = myPointTypes[cnt];// PointTypes.Diamond;
        fmt4.PointSize = 0.5;
        

        var xArr_cf = ReferenceDatacf[0].ExtractSubArrayShallow(-1, 0).To1DArray();
        var yArr_cf = ReferenceDatacf[0].ExtractSubArrayShallow(-1, 1).To1DArray();
        var xArr_nu = ReferenceDataNu[0].ExtractSubArrayShallow(-1, 0).To1DArray();
        var yArr_nu = ReferenceDataNu[0].ExtractSubArrayShallow(-1, 1).To1DArray();
        double scalecf = 0.3;        
        if(Re == 400)
        scalecf = 0.2;
         else if(Re == 700)
         scalecf =0.18;
         else if(Re == 1000)
         scalecf =0.0;

         double scalenu = 0.3;        
         if(Re == 400)
         scalenu = 0.92;
          else if(Re == 700)
          scalenu =0.92;
          else if(Re == 1000)
          scalenu =0.92;

        for(int i = 0;i < yArr_cf.Length;i++){
            yArr_cf[i]*=scalecf;
        }

        for(int i = 0;i < yArr_nu.Length;i++){
            yArr_nu[i]*=scalenu;
        }


        plotcf.AddDataGroup(xArr_cf,yArr_cf,fmt4);
        plotcf.XrangeMin=xCoordinates.Min();
        plotcf.XrangeMax=xCoordinates.Max();

        plotnuss.AddDataGroup( xArr_nu,yArr_nu, fmt4);
        plotnuss.XrangeMin=xCoordinates.Min();
        plotnuss.XrangeMax=xCoordinates.Max();
    }

        cnt++; 
    }
    PlotTableCfs[0, counter] = plotcf;
    PlotTableNusselts[0, counter] = plotnuss;

    counter++;



}
var gp = PlotTableCfs.ToGnuplot();
gp.PlotSVG(xRes:2000,yRes:400)


Error: System.Collections.Generic.KeyNotFoundException: The given key '(2, 700)' was not present in the dictionary.
   at System.Collections.Generic.Dictionary`2.get_Item(TKey key)
   at Submission#42.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
var gp = PlotTableNusselts.ToGnuplot();
gp.PlotSVG(xRes:2000,yRes:400)

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 0 
 
 
 
 
 5 
 
 
 
 
 10 
 
 
 
 
 15 
 
 
 
 
 20 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ER=2 
 
 
 ER=2 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M492.9,61.1 L546.3,61.1 M60.5,346.1 L61.1,346.1 L61.6,346.1 L62.2,346.1 L62.7,346.0 L63.3,346.0
 L63.8,346.0 L64.4,345.9 L64.9,345.8 L65.5,345.8 L66.0,345.7 L66.6,345.6 L67.1,345.4 L67.7,345.3
 L68.2,345.1 L68.8,344.9 L69.3,344.7 L69.9,344.4 L70.4,344.2 L71.0,343.9 L71.5,343.6 L72.1,343.4
 L72.6,343.0 L73.2,342.7 L73.7,342.4 L74.3,342.1 L74.8,341.7 L75.4,341.4 L75.9,341.0 L76.5,340.7
 L77.0,340.3 L77.6,340.0 L78.1,339.6 L78.7,339.2 L79.2,338.9 L79.8,338.5 L80.3,338.2 L80.9,337.8
 L81.4,337.5 L82.0,337.1 L82.5,336.8 L83.1,336.5 L83.6,336.1 L84.2,335.8 L84.7,335.5 L85.3,335.2
 L85.8,334.9 L86.4,334.6 L86.9,334.3 L87.5,333.9 L88.0,333.7 L88.6,333.4 L89.1,333.1 L89.7,332.8
 L90.2,332.5 L90.8,332.2 L91.3,331.9 L91.9,331.7 L92.4,331.4 L93.0,331.1 L93.5,330.8 L94.1,330.6
 L94.6,330.3 L95.2,330.1 L95.7,329.8 L96.3,329.6 L96.8,329.3 L97.4,329.0 L97.9,328.8 L98.5,328.6
 L99.0,328.3 L99.6,328.1 L100.1,327.8 L100.7,327.6 L101.2,327.4 L101.8,327.1 L102.3,326.9 L102.9,326.6
 L103.4,326.4 L104.0,326.2 L104.5,326.0 L105.1,325.7 L105.6,325.5 L106.2,325.3 L106.7,325.1 L107.3,324.8
 L107.8,324.6 L108.4,324.4 L108.9,324.2 L109.5,324.0 L110.0,323.8 L110.6,323.6 L111.1,323.3 L111.7,323.1
 L112.2,322.9 L112.8,322.7 L113.3,322.5 L113.9,322.3 L114.4,322.1 L115.0,321.9 L115.5,321.7 L116.1,321.5
 L116.6,321.3 L117.2,321.1 L117.7,320.9 L118.3,320.7 L118.8,320.5 L119.4,320.3 L119.9,320.1 L120.5,319.9
 L121.0,319.7 L121.6,319.5 L122.1,319.3 L122.7,319.1 L123.2,318.9 L123.8,318.7 L124.3,318.6 L124.9,318.4
 L125.4,318.2 L126.0,318.0 L126.5,317.8 L127.1,317.6 L127.6,317.4 L128.2,317.2 L128.7,317.1 L129.3,316.9
 L129.8,316.7 L130.4,316.5 L130.9,316.3 L131.5,316.1 L132.0,316.0 L132.6,315.8 L133.1,315.6 L133.7,315.4
 L134.2,315.2 L134.8,315.1 L135.3,314.9 L135.9,314.7 L136.4,314.5 L137.0,314.3 L137.5,314.2 L138.1,314.0
 L138.6,313.8 L139.2,313.6 L139.7,313.5 L140.3,313.3 L140.8,313.1 L141.4,312.9 L141.9,312.8 L142.5,312.6
 L143.0,312.4 L143.6,312.3 L144.1,312.1 L144.7,311.9 L145.2,311.7 L145.8,311.6 L146.3,311.4 L146.9,311.2
 L147.4,311.1 L148.0,310.9 L148.5,310.7 L149.1,310.5 L149.6,310.4 L150.2,310.2 L150.7,310.0 L151.3,309.9
 L151.8,309.7 L152.4,309.5 L152.9,309.4 L153.5,309.2 L154.0,309.0 L154.6,308.9 L155.1,308.7 L155.7,308.5
 L156.2,308.4 L156.8,308.2 L157.3,308.0 L157.9,307.9 L158.4,307.7 L159.0,307.5 L159.5,307.4 L160.1,307.2
 L160.6,307.0 L161.2,306.9 L161.7,306.7 L162.3,306.5 L162.8,306.4 L163.4,306.2 L163.9,306.1 L164.5,305.9
 L165.0,305.7 L165.6,305.6 L166.1,305.4 L166.7,305.2 L167.2,305.1 L167.8,304.9 L168.3,304.7 L168.9,304.6
 L169.4,304.4 L170.0,304.3 L170.5,304.1 L171.1,303.9 L171.6,303.8 L172.2,303.6 L172.7,303.4 L173.3,303.3
 L173.8,303.1 L174.4,302.9 L174.9,302.8 L175.5,302.6 L176.0,302.4 L176.6,302.3 L177.1,302.1 L177.7,302.0
 L178.2,301.8 L178.8,301.6 L179.3,301.5 L179.9,301.3 L180.4,301.1 L181.0,301.0 L181.5,300.8 L182.1,300.6
 L182.6,300.5 L183.2,300.3 L183.7,300.1 L184.3,300.0 L184.8,299.8 L185.4,299.7 L185.9,299.5 L186.5,299.3
 L187.0,299.2 L187.6,299.0 L188.1,298.8 L188.7,298.7 L189.2,298.5 L189.8,298.3 L190.3,298.2 L190.9,298.0
 L191.4,297.8 L192.0,297.7 L192.5,297.5 L193.1,297.3 L193.6,297.1 L194.2,297.0 L194.7,296.8 L195.3,296.6
 L195.8,296.5 L196.4,296.3 L196.9,296.1 L197.5,296.0 L198.0,295.8 L198.6,295.6 L199.1,295.5 L199.7,295.3
 L200.2,295.1 L200.8,294.9 L201.3,294.8 L201.9,294.6 L20

Points of separation are obtained by finding the positions where the value of the gradient of velocityX normal to the wall changes its sign. 

In [ ]:
ReattachmentPositionsTOP

key,value
"(2, 200)",[ ]
"(2, 250)","[ 8.51916343540012, 12.329609712614705 ]"
"(2, 300)","[ 8.938770309901487, 15.111056992206267 ]"
"(2, 350)","[ 9.395943066476896, 17.57079780908193 ]"
"(2, 400)","[ 9.854436725989947, 19.81351305071657 ]"


In [ ]:
List<(double,double)> primaryReatLengths = new List<(double,double)>();
List<double> Rey = new List<double>();
List<double> Lenghths = new List<double>();

//Try to filter values in order to obtain the length of reattachment after the aparition of the primary vortex  
foreach(var a in ReattachmentPositions){
        double Reynolds = a.Key.Item2;
        double[] allLengths = a.Value;
        try{
            double primLength = allLengths[0];
            if(primLength < 0.3)
                primLength = allLengths[1];
Rey.Add(Reynolds);
Lenghths.Add(primLength);
            primaryReatLengths.Add((Reynolds,primLength));
        }catch(Exception e){
        continue;
        }
}

double[] Re_arr = Rey.ToArray();
double[] LeArr = Lenghths.ToArray();

Comparison of reattachment lengths for the primary vortex for ER =1.9423 

In [ ]:
var plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines;

fmt.LineColor = myCollors[0];
plot.AddDataGroup("BoSSS", Re_arr, LeArr, fmt);

// Reference data
MultidimensionalArray[] ReferenceDataNu = new MultidimensionalArray[1];

string path2 = Path.Combine(Directory.GetCurrentDirectory(), @"reatlengths_bot.txt"); // CF_RE400_ER1data for ER 1.94...
ReferenceDataNu[0] = IMatrixExtensions.LoadFromTextFile(path2);
var fmt4 = new PlotFormat();
fmt4.Style = Styles.Points;

fmt4.PointType = myPointTypes[0];// PointTypes.Diamond;
fmt4.PointSize = 0.5;
var xArr_nu = ReferenceDataNu[0].ExtractSubArrayShallow(-1, 0).To1DArray();
for(int i = 0; i < xArr_nu.Length; i++){
    xArr_nu[i]*=0.5; // REescale Reynolds for comparison with paper 
}

var yArr_nu = ReferenceDataNu[0].ExtractSubArrayShallow(-1, 1).To1DArray();
plot.AddDataGroup("reference",xArr_nu,yArr_nu,fmt4);
plot . PlotNow ()


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 2 
 
 
 
 
 4 
 
 
 
 
 6 
 
 
 
 
 8 
 
 
 
 
 10 
 
 
 
 
 12 
 
 
 
 
 14 
 
 
 
 
 -50 
 
 
 
 
 0 
 
 
 
 
 50 
 
 
 
 
 100 
 
 
 
 
 150 
 
 
 
 
 200 
 
 
 
 
 250 
 
 
 
 
 300 
 
 
 
 
 350 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 BoSSS 
 
 
 BoSSS 
 
 
 
 
 
 reference 
 
 
 reference

In [ ]:
var plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines;

fmt.LineColor = myCollors[0];
plot.AddDataGroup("BoSSS_Deatach", Re_arr, LeArr, fmt);

// Reference data
MultidimensionalArray[] ReferenceDataNu = new MultidimensionalArray[1];
string path2 = Path.Combine(Directory.GetCurrentDirectory(), @"reatlengths_bot.txt"); // CF_RE400_ER1data for ER 1.94...
ReferenceDataNu[0] = IMatrixExtensions.LoadFromTextFile(path2);
var fmt4 = new PlotFormat();
fmt4.Style = Styles.Points;

fmt4.PointType = myPointTypes[0];// PointTypes.Diamond;
fmt4.PointSize = 0.5;
var xArr_nu = ReferenceDataNu[0].ExtractSubArrayShallow(-1, 0).To1DArray();
for(int i = 0; i < xArr_nu.Length; i++){
    xArr_nu[i]*=0.5; // REescale Reynolds for comparison with paper 
}

var yArr_nu = ReferenceDataNu[0].ExtractSubArrayShallow(-1, 1).To1DArray();
plot.AddDataGroup("reference",xArr_nu,yArr_nu,fmt4);
plot . PlotNow ()


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 2 
 
 
 
 
 4 
 
 
 
 
 6 
 
 
 
 
 8 
 
 
 
 
 10 
 
 
 
 
 12 
 
 
 
 
 14 
 
 
 
 
 -50 
 
 
 
 
 0 
 
 
 
 
 50 
 
 
 
 
 100 
 
 
 
 
 150 
 
 
 
 
 200 
 
 
 
 
 250 
 
 
 
 
 300 
 
 
 
 
 350 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 BoSSSDeatach 
 
 
 BoSSS D eatach 
 
 
 
 
 
 reference 
 
 
 reference